# Kaggle API Credential File Upload

In [ ]:
# !pip install -q kaggle
from google.colab import files
files.upload()

# Load and Extract Dataset

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c 30-days-of-ml

 82% 33.0M/40.3M [00:00<00:00, 75.2MB/s]
100% 40.3M/40.3M [00:00<00:00, 91.0MB/s]
  0% 0.00/470k [00:00<?, ?B/s]
100% 470k/470k [00:00<00:00, 158MB/s]
 36% 9.00M/25.2M [00:00<00:00, 34.0MB/s]
100% 25.2M/25.2M [00:00<00:00, 62.9MB/s]


In [ ]:
!mkdir train
!unzip train.csv.zip -d train

!mkdir test
!unzip test.csv.zip -d test

!mkdir sample_submission
!unzip sample_submission.csv.zip -d sample_submission

Archive:  train.csv.zip
  inflating: train/train.csv         
Archive:  test.csv.zip
  inflating: test/test.csv           
Archive:  sample_submission.csv.zip
  inflating: sample_submission/sample_submission.csv  


# Import Libraries

In [ ]:
# Warning Libraries 
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)

# Scientific and Data Manipulation Libraries 
import pandas as pd
import numpy as np
import math
import gc
import os


# Data Preprocessing, Machine Learning and Metrics Libraries 
from sklearn.preprocessing            import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing            import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble                 import VotingRegressor
from sklearn.metrics                  import mean_squared_error

from sklearn.feature_selection        import RFECV
# model visualization
# !pip install shap
# import shap


# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.model_selection import KFold, RepeatedKFold, train_test_split
from sklearn.pipeline import Pipeline

# Boosting Algorithms 
from xgboost                          import XGBRegressor
!pip install catboost
# from catboost                         import CatBoostRegressor
from lightgbm                         import LGBMRegressor

from sklearn.model_selection import GridSearchCV


# Data Visualization Libraries 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px

     |████████████████████████████████| 67.4 MB 26 kB/s 


# EDA

In [ ]:
# read data

train = pd.read_csv('/content/train/train.csv')
test = pd.read_csv('/content/test/test.csv')

sub = pd.read_csv('/content/sample_submission/sample_submission.csv')

# Looks at the first 5 rows of the Train and Test data
display('Train Head :',train.head())
display('Test Head :',test.head())


# Displays Information of Columns of Train and Test data
train.info()
test.info()


# Display Descriptive Statistics of Train and Test data
display('Train Description :',train.describe())
display('Test  Description :',test.describe())


# Displays Correlation between Features through HeatMap - Ligther Color means Higher Correlation
# sns.heatmap(train.corr(), annot = True)

'Train Head :'

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


'Test Head :'

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 26 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cont0   300000 non-null  float64
 12  cont1   300000 non-null  float64
 13  cont2   300000 non-null  float64
 14  cont3   300000 non-null  float64
 15  cont4   300000 non-null  float64
 16  cont5   300000 non-null  float64
 17  cont6   300000 non-null  float64
 18  cont7   300000 non-null  float64
 19  cont8   300000 non-null  float64
 20  cont9   300000 non-null  float64
 21  cont10  30

'Train Description :'

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,250018.576947,0.527335,0.460926,0.490498,0.496689,0.491654,0.510526,0.467476,0.537119,0.498456,0.474872,0.474492,0.473216,0.494561,0.508273,8.241979
std,144450.150010,0.230599,0.214003,0.253346,0.219199,0.240074,0.228232,0.210331,0.218140,0.239920,0.218007,0.255949,0.222022,0.247292,0.222950,0.746555
min,1.000000,-0.118039,-0.069309,-0.056104,0.130676,0.255908,0.045915,-0.224689,0.203763,-0.260275,0.117896,0.048732,0.052608,-0.074208,0.151050,0.140329
25%,124772.500000,0.405965,0.310494,0.300604,0.329783,0.284188,0.354141,0.342873,0.355825,0.332486,0.306874,0.276017,0.308151,0.289074,0.300669,7.742071
50%,250002.500000,0.497053,0.427903,0.502462,0.465026,0.390470,0.488865,0.429383,0.504661,0.439151,0.434620,0.459975,0.433812,0.422887,0.472400,8.191373
75%,375226.500000,0.668060,0.615113,0.647512,0.664451,0.696599,0.669625,0.573383,0.703441,0.606056,0.614333,0.691579,0.642057,0.714502,0.758447,8.728634
max,499999.000000,1.058443,0.887253,1.034704,1.039560,1.055424,1.067649,1.111552,1.032837,1.040229,0.982922,1.055960,1.071444,0.975035,0.905992,10.411992


'Test  Description :'

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,249970.884580,0.526858,0.460882,0.491686,0.496263,0.492200,0.509944,0.468050,0.537617,0.497587,0.474630,0.473625,0.473589,0.492756,0.508303
std,144169.245904,0.230506,0.213896,0.252852,0.219304,0.240251,0.228100,0.210167,0.218197,0.239337,0.217558,0.255307,0.221844,0.247105,0.222977
min,0.000000,-0.117993,-0.069316,-0.056095,0.130655,0.255903,0.045921,-0.224613,0.203763,-0.260236,0.117894,0.048734,0.052615,-0.074201,0.151052
25%,125316.750000,0.405787,0.310536,0.302189,0.328865,0.284188,0.353682,0.342813,0.356482,0.332762,0.306940,0.274991,0.308495,0.288094,0.300697
50%,249997.000000,0.496602,0.427656,0.503075,0.464590,0.395080,0.488378,0.429819,0.505216,0.438716,0.434610,0.460042,0.433857,0.422065,0.472446
75%,374649.750000,0.666389,0.615041,0.648234,0.663401,0.697252,0.668586,0.573861,0.703289,0.605368,0.610768,0.690143,0.642201,0.711038,0.758496
max,499995.000000,1.058434,0.887253,1.034687,1.039558,1.055423,1.067641,1.111540,1.032823,1.040208,0.982928,1.055955,1.071445,0.975032,0.906013


In [ ]:
display(train[train.duplicated(keep=False)])

display(test[test.duplicated(keep=False)])

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13


In [ ]:
train.isna().sum().sum(), test.isna().sum().sum()

(0, 0)

# Data Transformation

In [ ]:
# ML Dataset
X=train.drop(['target','id'],axis=1)
y=train['target']

# test dataset
test =  test.drop('id',axis=1)

# determine categorical and numerical features
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns


ordinal_encoder = OrdinalEncoder()
X[categorical_ix] = ordinal_encoder.fit_transform(X[categorical_ix])
test[categorical_ix] = ordinal_encoder.transform(test[categorical_ix])

# one-hot encoding
# one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# X[categorical_ix] = one_hot_encoder.fit_transform(X[categorical_ix])
# test[categorical_ix] = one_hot_encoder.transform(test[categorical_ix])

# # Scaling features
scaler = RobustScaler() # StandardScaler()  RobustScaler()  MinMaxScaler() MaxAbsScaler()
X[numerical_ix] = scaler.fit_transform(X[numerical_ix])
test[numerical_ix] = scaler.transform(test[numerical_ix])

In [ ]:
X.shape , y.shape, test.shape

((300000, 24), (300000,), (200000, 24))

# OOF

In [ ]:
# https://machinelearningmastery.com/out-of-fold-predictions-in-machine-learning/
# https://www.kaggle.com/miguelquiceno/30-days-kfold-xgboost#Modeling
# https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/


hyperparams = {'depth': 6, 'iterations': 200, 'l2_leaf_reg': 3, 'learning_rate': 0.1,'loss_function':'RMSE','eval_metric':"RMSE", 'verbose' : False}

SEED = 299792458
N_ESTIMATORS = 10000
ctb_params = {
    'bootstrap_type': 'Poisson',
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'random_seed': SEED,
    'task_type': 'GPU',
    'max_depth': 8,
    'learning_rate': 5e-3,
    'n_estimators': N_ESTIMATORS,
    'max_bin': 280,
    'min_data_in_leaf': 64,
    'l2_leaf_reg': 0.01,
    'subsample': 0.8,
    'verbose' : False
}
kf = KFold(n_splits=10, shuffle=True , random_state=42)

test_preds = 0
mean_rmse = 0
fold_preds = np.zeros((X.shape[0],))

for num, (train_id, valid_id) in enumerate(kf.split(X)):
  X_train, y_train = X.loc[train_id], y.loc[train_id]
  X_valid, y_valid = X.loc[valid_id], y.loc[valid_id]

  eval_set = [(X_valid, y_valid)]

  # model = CatBoostRegressor(eval_metric="RMSE", verbose = False)

  # Define the model 
  model = XGBRegressor(n_estimators=2000, # 1375
                       max_depth = 3,
                       learning_rate=0.14, 
                       olsample_bytree= 0.5,
                       subsample=0.99, 
                       random_state=1, reg_alpha = 25.4)
  model.fit(X_train, y_train, early_stopping_rounds = 100, eval_set=eval_set, verbose=False)

  # model = CatBoostRegressor(**ctb_params)
  # model.fit(X_train, y_train, eval_set=eval_set)

  test_preds+=model.predict(test) / 10

  fold_preds[valid_id] = model.predict(X_valid)
  fold_rmse = mean_squared_error(y_valid, fold_preds[valid_id], squared=False)
  print(f"\n{num+1} --- RMSE : {fold_rmse}")

  mean_rmse += fold_rmse/10

print(f"Mean RMSE : {mean_rmse}")

# Mean RMSE : 0.7203118094780596 , catboost base model
# Mean RMSE : 0.7169277566773257 , XGB with hyperparams set+ RobustScaler , competition : 0.71835, 2nd submission 0.71812
# Mean RMSE : XGB with hyperparams set (N-estimators - 2000)+ RobustScaler, competition - 0.71803

[09:08:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

1 --- RMSE : 0.7166679351283799
[09:20:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

2 --- RMSE : 0.7159528719053435
[09:29:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

3 --- RMSE : 0.7154653965398875
[09:39:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

4 --- RMSE : 0.7176289068947505
[09:49:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

5 --- RMSE : 0.7218356630425924
[10:00:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

6 --- RMSE : 0.7151275543556598
[10:09:30] WARNING: /workspace/src/objec

In [ ]:
# parameters = {'depth'         : [6,8,10],
#                   'learning_rate' : [0.01, 0.05, 0.1],
#                   'iterations'    : [30, 50, 100]
#                  }
parameters = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}



grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

print("Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

## Predict Test dataset

In [ ]:
test_preds = model.predict(test)

display(test_preds)

sub.iloc[:, 1:] = test_preds

# array([8.0396805, 8.3653   , 8.386101 , ..., 8.496086 , 8.1603985,
       7.9626327], dtype=float32)

array([8.0396805, 8.3653   , 8.386101 , ..., 8.496086 , 8.1603985,
       7.9626327], dtype=float32)

## OOF test predicts

In [ ]:
sub.target = test_preds
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)
sub.shape

(200000, 2)

In [ ]:
!kaggle competitions submit -c 30-days-of-ml -f submission.csv -m ""

100% 3.01M/3.01M [00:01<00:00, 1.84MB/s]
Successfully submitted to 30 Days of ML